<a href="https://colab.research.google.com/github/ameek2/CISC662/blob/master/Benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hi y'all. I'm going to use this to put some of the benchmark code in.
The below code is taken from Google's notebook introducing TPUs. All it does is check whether or not you have a TPU selected as your hardware accelerator. 
To set it, go to Runtime->Change runtime settings.

In [ ]:
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

Credit:

1: https://github.com/EpistasisLab/pmlb

2: https://bitbucket.org/berkeleylab/cs-roofline-toolkit/src/master/Empirical_Roofline_Tool-1.1.0/ert 

Currently going through here and pulling some scripts to run. I'll put them in the code boxes below. For now these will just be some samples from the repo's I'm using as references. We'll pull them together afterward.

First is the core loop from perfzero. We can pull different algos from Pytorch and loop over them. Probably one at a time when we analyze in ERT though.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sb

from pmlb import fetch_data, classification_dataset_names

logit_test_scores = []
gnb_test_scores = []

for classification_dataset in classification_dataset_names:
    X, y = fetch_data(classification_dataset, return_X_y=True)
    train_X, test_X, train_y, test_y = train_test_split(X, y)

    logit = LogisticRegression()
    gnb = GaussianNB()

    logit.fit(train_X, train_y)
    gnb.fit(train_X, train_y)

    logit_test_scores.append(logit.score(test_X, test_y))
    gnb_test_scores.append(gnb.score(test_X, test_y))

sb.boxplot(data=[logit_test_scores, gnb_test_scores], notch=True)
plt.xticks([0, 1], ['LogisticRegression', 'GaussianNB'])
plt.ylabel('Test Accuracy')

In [ ]:
#!/usr/bin/env python3

# Import builtin Python modules
import sys,os.path

# Import ERT python modules
from Python.ert_core  import ert_core
from Python.ert_utils import *

# Get the path to the directory where this was run
exe_path = os.path.dirname(sys.argv[0])
if exe_path == "":
  exe_path = "."

# Create the ERT object
ert = ert_core()

# Initialize the execution path
ert.exe_path = exe_path

# Parse the command line
if ert.flags() != 0:
  sys.stderr.write("\n--- Parsing command line arguments failed --- \n\n")
  sys.exit(1)
sys.stdout.flush()

# Configure the ERT using the configuration file given on the command line
if ert.configure() != 0:
  sys.stderr.write("\n--- Configuring ERT failed ---\n\n")
  sys.exit(1)
sys.stdout.flush()

# Get the list of flops/element to try (specified by the user)
flops_list = parse_int_list(ert.dict["CONFIG"]["ERT_FLOPS"][0])

# Go through the flops/element list and process each case
for flop in flops_list:
  # A bit of output if any verbosity has been requested
  if ert.options.verbose > 0:
    print("FLOP count %d..." % flop)
    sys.stdout.flush()

  # Set the ERT object flops/element
  ert.flop = flop

  # Create a directory for these experiments (and give it to the ERT object)
  ert.flop_dir = "%s/FLOPS.%03d" % (ert.results_dir,flop)
  make_dir_if_needed(ert.flop_dir,"run",ert.options.verbose > 1)

  # Build the current code with the current flops/element
  if ert.build() != 0:
    sys.stderr.write("\n--- Building ERT failed ---\n\n")
    sys.exit(1)
  sys.stdout.flush()

  # Run the built code over a variety of MPI, OpenMP, and/or CUDA
  # configurations specified by the user
  if ert.run() != 0:
    sys.stderr.write("\n--- Running ERT failed ---\n\n")
    sys.exit(1)
  sys.stdout.flush()

  # Process all the results for the current flops/element
  if ert.process() != 0:
    sys.stderr.write("\n--- Processing ERT results failed ---\n\n")
    sys.exit(1)
  sys.stdout.flush()

  # Make graphs of all the results for the current flops/element
  if ert.graphs() != 0:
    sys.stderr.write("\n--- Making ERT individual graphs failed ---\n\n")
    sys.exit(1)
  sys.stdout.flush()

  if ert.options.verbose > 0:
    print()

# When all the experiments have been processed, generate the roofline graph
# and JSON output
if ert.roofline() != 0:
  sys.stderr.write("\n--- Making ERT roofline failed ---\n\n")
  sys.exit(1)

# All done
sys.stdout.flush()
sys.exit(0)

Here are some scripts to determine the specs of the GPU/TPU hardware accelerator options. Print these before running, and then we can know where to place them on the roofline model graphs. 

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 18143965305416487764
 xla_global_id: -1, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14415560704
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 981142621764829025
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
 xla_global_id: 416903419]